In [1]:
from pyteaser import Summarize, SummarizeUrl
from pprint import pprint
import pandas as pd
import numpy as np
import tagui as t
import sqlite3
from sqlite3 import Error
from datetime import datetime

In [2]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("DB Connection setup")
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn
        else:
            print("DB Connection failed")


if __name__ == '__main__':
    conn = create_connection("./db/news.db")

DB Connection setup


In [3]:
def getSiteByName(name):
    cur = conn.cursor()
    cur.execute("SELECT * FROM site where name=?",(name,))
    rows = cur.fetchall()
    return rows

In [13]:
site = getSiteByName("StraitsTimes")
site_url = site[0][2]
site_id = site[0][0]

In [14]:
t.init(visual_automation = True, chrome_browser = True)
t.url(site_url)

[RPA][ERROR] - use close() before using init() again


True

In [15]:
number_bb = t.count('(//div[contains(@data-vr-zone, "Top Stories")]//span[contains(@class, "story-headline")])')

df_bb = pd.DataFrame(index=range(0,number_bb-2), columns = ['Sno', 'Title', 'URL', 'Summary','Img_URL'])


for n in range(0, number_bb-2):
    title=t.read('//div[contains(@data-vr-zone, "Top Stories {}")]//span[contains(@class, "story-headline")]'.format(n))
    URL_b=t.read('//div[contains(@data-vr-zone, "Top Stories {}")]//span[contains(@class, "story-headline")]//@href'.format(n))
    URL = "https://www.straitstimes.com/" + str(URL_b)
    Img_URL = t.read('//div[contains(@data-vr-zone, "Top Stories {}")]//span[contains(@class, "story-headline")]/ancestor::div[contains(@class, "body")]/..//img/@src'.format(n))
    summaries = SummarizeUrl(URL)
    
    df_bb.iloc[n, 0] = n
    df_bb.iloc[n, 1] = title
    df_bb.iloc[n, 2] = URL
    df_bb.iloc[n, 3] = summaries
    df_bb.iloc[n, 4] = Img_URL
    
t.close()

[RPA][ERROR] - cannot find //div[contains(@data-vr-zone, "Top Stories 8")]//span[contains(@class, "story-headline")]/ancestor::div[contains(@class, "body")]/..//img/@src


True

In [16]:
def remove_str(summm_l, SS):
    summm = ''.join(summm_l)
    summm = summm.encode('utf-8').decode('utf-8', 'ignore')
    if SS in summm:
        summm = summm.replace(SS, '')
    
    return summm

for i in range(0, df_bb.shape[0]):
    df_bb['Summary'][i] = remove_str(df_bb['Summary'][i], "u'")

In [17]:
def insertNews(conn,news):    
    sql = ''' INSERT INTO news(site_id,create_date,title,url,summary,img_url,latest)
              VALUES(?,?,?,?,?,?,1) '''
    cur = conn.cursor()
    cur.execute(sql, news)
    return cur.lastrowid

In [18]:
def markNews(conn, site_id):
    cur = conn.cursor()
    cur.execute("UPDATE news set latest=0 where latest=1 and site_id=? ",(site_id,))

In [19]:
now = datetime.now()
# dd/mm/YY
dt_string = now.strftime("%d/%m/%Y")
markNews(conn, site_id)
for index, row in df_bb.iterrows():
    title = row['Title']
    url = row['URL']
    summary = row['Summary']
    img_url = row['Img_URL']
    news = (site_id,dt_string,title,url,summary,img_url)
    news_id = insertNews(conn,news)
    print("News %d has been created" % news_id)
    
conn.commit()

News 97 has been created
News 98 has been created
News 99 has been created
News 100 has been created
News 101 has been created
News 102 has been created
News 103 has been created
News 104 has been created
News 105 has been created


In [15]:
conn.close()